## TÍNH TOÁN VÀ VẼ BIỂU ĐỒ CỦA TRANG BỊ TƯỚNG VÀ LÕI NÂNG CẤP  
### _(Dựa trên tỉ lệ sử dụng, tính toán thêm vị trí trung bình, tỉ lệ top1, top 4, có các biểu độ tròn, cột)_

In [23]:
import pandas as pd
import numpy as np
import ast
from itertools import chain


#### Đọc dữ liệu lấy data về champion và lõi

In [24]:
'''
Lọc trùng theo mã trận đấu và danh sách đồ vì:
* Các người chơi sẽ gặp nhau ở các trận đấu nên mã trận đấu sẽ bị trùng giữa các thư mục ngườichơi
* Có 127 trang bị chia thành 4 loại, mỗi champion có tối đa 3 trang bị chưa kể 1 trận có từ 8-10 trang bị và 6-9 champion 
mà mỗi champion có thể để từ 0-3 trang bị nên số trường hợp tối đa là rất lớn mà dữ liệu đầu vào chỉ có 32000 đội hình nên các bộ trang bị
sẽ không thể bị trùng giữa 2 người chơi khác nhau được!
'''
item_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols=[
                      'ID Match', 'List Name Item', 'Champion']).drop_duplicates().reset_index(drop=True)
# item_df = pd.read_csv('process_data_tft_each_item_csv.csv').drop_duplicates()


In [25]:
item_df
print(item_df)
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(item_df)


             ID Match                                           Champion  \
0       KR_6267320509  ['TFT8_Gangplank', 'TFT8_Vi', 'TFT8_LeeSin', '...   
1       KR_6267320509  ['TFT8_Lux', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT8...   
2       KR_6267320509  ['TFT8_Galio', 'TFT8_Nasus', 'TFT8_Gangplank',...   
3       KR_6267320509  ['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...   
4       KR_6267320509  ['TFT8_Lulu', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT...   
...               ...                                                ...   
18091  NA1_4523403727  ['TFT8_Blitzcrank', 'TFT8_Renekton', 'TFT8_Vi'...   
18092  NA1_4523403727  ['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...   
18093  NA1_4523403727  ['TFT8_WuKong', 'TFT8_Jax', 'TFT8_Zoe', 'TFT8_...   
18094  NA1_4523403727  ['TFT8_Kayle', 'TFT8_Gangplank', 'TFT8_Fiora',...   
18095  NA1_4523403727  ['TFT8_Fiora', 'TFT8_Vi', 'TFT8_Alistar', 'TFT...   

                                          List Name Item  
0      [['TFT_Item_ZekesHera

In [26]:
list_items = []
for i in range(len(item_df['Champion'])):
    list_items.append(ast.literal_eval(item_df['Champion'][i]))
list_items

[['TFT8_Gangplank',
  'TFT8_Vi',
  'TFT8_LeeSin',
  'TFT8_Malphite',
  'TFT8_Ezreal',
  'TFT8_Sona',
  'TFT8_Kaisa'],
 ['TFT8_Lux',
  'TFT8_Annie',
  'TFT8_Yuumi',
  'TFT8_Sona',
  'TFT8_Alistar',
  'TFT8_Ekko',
  'TFT8_Taliyah',
  'TFT8_Syndra'],
 ['TFT8_Galio',
  'TFT8_Nasus',
  'TFT8_Gangplank',
  'TFT8_LeeSin',
  'TFT8_Malphite',
  'TFT8_Yuumi',
  'TFT8_Alistar',
  'TFT8_Nunu'],
 ['TFT8_Fiora',
  'TFT8_Yasuo',
  'TFT8_Vayne',
  'TFT8_Nilah',
  'TFT8_Zoe',
  'TFT8_Leblanc',
  'TFT8_Zed',
  'TFT8_Sejuani'],
 ['TFT8_Lulu',
  'TFT8_Annie',
  'TFT8_Yuumi',
  'TFT8_Sona',
  'TFT8_Alistar',
  'TFT8_Zoe',
  'TFT8_Ekko',
  'TFT8_Soraka'],
 ['TFT8_Talon',
  'TFT8_Blitzcrank',
  'TFT8_Vi',
  'TFT8_Camille',
  'TFT8_Zoe',
  'TFT8_Leblanc',
  'TFT8_Viego',
  'TFT8_Soraka'],
 ['TFT8_Sivir',
  'TFT8_Velkoz',
  'TFT8_Chogath',
  'TFT8_Rammus',
  'TFT8_Rammus',
  'TFT8_Samira',
  'TFT8_AurelionSol',
  'TFT8_Urgot'],
 ['TFT8_Lulu',
  'TFT8_Annie',
  'TFT8_Yuumi',
  'TFT8_Alistar',
  'TFT8_Zoe',
  'T

In [27]:
def get_best_worst_items(list_items):
    print(len(list_items))
    flatten_list = list(chain.from_iterable(list_items))
    # flatten_list = list(chain.from_iterable(flatten_list))
    # print(flatten_list[:10])
    flatten_list = np.array(flatten_list, dtype = object)
    # print(flatten_list[:10])
    unique, counts = np.unique(flatten_list, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))

    best_item = list(b.items())
    worst_item = list(b.items())
    return best_item,worst_item

In [28]:
best_item,worst_item = get_best_worst_items(list_items)

18096


In [29]:
print(best_item, worst_item)


[('TFT8_Poppy', 193), ('TFT8_Sylas', 394), ('TFT8_Draven', 571), ('TFT8_Ashe', 602), ('TFT8_Kayle', 655), ('TFT8_Rell', 793), ('TFT8_Lulu', 875), ('TFT8_Jinx', 883), ('TFT8_Ezreal', 956), ('TFT8_BelVeth', 959), ('TFT8_Talon', 989), ('TFT8_Camille', 999), ('TFT8_WuKong', 1063), ('TFT8_MissFortune', 1107), ('TFT8_Mordekaiser', 1108), ('TFT8_Viego', 1136), ('TFT8_Senna', 1157), ('TFT8_Lux', 1184), ('TFT8_Sett', 1233), ('TFT8_AurelionSol', 1370), ('TFT8_Fiora', 1582), ('TFT8_Renekton', 1597), ('TFT8_Sivir', 1671), ('TFT8_Nasus', 1701), ('TFT8_Chogath', 1705), ('TFT8_Rammus', 1947), ('TFT8_Kaisa', 1965), ('TFT8_Velkoz', 1992), ('TFT8_Aphelios', 2113), ('TFT8_Zac', 2135), ('TFT8_Urgot', 2223), ('TFT8_Yasuo', 2285), ('TFT8_Syndra', 2326), ('TFT8_Taliyah', 2412), ('TFT8_Zed', 2414), ('TFT8_Samira', 2417), ('TFT8_Blitzcrank', 2520), ('TFT8_Nunu', 2572), ('TFT8_Galio', 2619), ('TFT8_Sona', 2751), ('TFT8_Fiddlesticks', 2817), ('TFT8_Jax', 2902), ('TFT8_Malphite', 3107), ('TFT8_Gangplank', 3193), 

#### Vị trí trung bình

In [30]:
average_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols=[
                         'ID Match','List Name Item','Champion', 'Placement']).drop_duplicates().reset_index(drop=True)


In [31]:
average_df


,ID Match,Champion,List Name Item,Placement
0,KR_6267320509,"['TFT8_Gangplank', 'TFT8_Vi', 'TFT8_LeeSin', '...","[['TFT_Item_ZekesHerald'], ['TFT_Item_BrambleV...",1
1,KR_6267320509,"['TFT8_Lux', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT8...","[['TFT8_Item_HeartEmblemItem'], ['TFT_Item_Bra...",4
2,KR_6267320509,"['TFT8_Galio', 'TFT8_Nasus', 'TFT8_Gangplank',...","[['TFT_Item_GargoyleStoneplate'], [], ['TFT_It...",7
3,KR_6267320509,"['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...","[[], ['TFT8_Item_HackerEmblemItem'], ['TFT_Ite...",2
4,KR_6267320509,"['TFT8_Lulu', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT...","[[], ['TFT_Item_WarmogsArmor', 'TFT_Item_Titan...",5
...,...,...,...,...
18091,NA1_4523403727,"['TFT8_Blitzcrank', 'TFT8_Renekton', 'TFT8_Vi'...","[[], ['TFT_Item_ArchangelsStaff', 'TFT_Item_Je...",6
18092,NA1_4523403727,"['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...","[['TFT_Item_TitanicHydra'], ['TFT_Item_ChainVe...",3
18093,NA1_4523403727,"['TFT8_WuKong', 'TFT8_Jax', 'TFT8_Zoe', 'TFT8_...","[[], [], ['TFT8_Item_HeartEmblemItem'], ['TFT_...",2
18094,NA1_4523403727,"['TFT8_Kayle', 'TFT8_Gangplank', 'TFT8_Fiora',...","[[], [], [], [], ['TFT_Item_StatikkShiv', 'TFT...",7


In [32]:
def convert_string_to_list(ok, placement, column):
    temp = ok[ok.eq(placement).any(1)].reset_index(drop=True)
    top = []
    for i in range(len(temp)):
        top.append(ast.literal_eval(temp[column][i]))
    return top


In [33]:
average_df[average_df.eq(1).any(1)]['Champion'].to_csv('Champion_top1.csv')



In [34]:
item_on_top = {}
for i in range(1, 9, 1):
    temp = convert_string_to_list(average_df, i, 'Champion')
    temp1 = 'top_' + str(i)
    item_on_top[temp1] = get_best_worst_items(temp)


2262
2262
2262
2262
2262
2262
2262
2262


In [35]:
best_item_top_1, worst_item_top_1 = item_on_top['top_1']
print(best_item_top_1, "\n", worst_item_top_1)

[('TFT8_Poppy', 12), ('TFT8_Sylas', 41), ('TFT8_Ashe', 48), ('TFT8_Kayle', 59), ('TFT8_Rell', 63), ('TFT8_Draven', 70), ('TFT8_Jinx', 74), ('TFT8_Lulu', 79), ('TFT8_BelVeth', 90), ('TFT8_Lux', 110), ('TFT8_MissFortune', 117), ('TFT8_Talon', 117), ('TFT8_Senna', 120), ('TFT8_AurelionSol', 129), ('TFT8_Camille', 134), ('TFT8_Ezreal', 135), ('TFT8_WuKong', 144), ('TFT8_Fiora', 145), ('TFT8_Sett', 155), ('TFT8_Viego', 155), ('TFT8_Chogath', 179), ('TFT8_Zed', 179), ('TFT8_Yasuo', 187), ('TFT8_Renekton', 195), ('TFT8_Kaisa', 196), ('TFT8_Mordekaiser', 204), ('TFT8_Zac', 206), ('TFT8_Rammus', 211), ('TFT8_Sivir', 221), ('TFT8_Velkoz', 260), ('TFT8_Taliyah', 268), ('TFT8_Vayne', 289), ('TFT8_Nilah', 298), ('TFT8_Samira', 312), ('TFT8_Nasus', 322), ('TFT8_Blitzcrank', 343), ('TFT8_Aphelios', 370), ('TFT8_Annie', 385), ('TFT8_Sona', 385), ('TFT8_Jax', 389), ('TFT8_Leblanc', 417), ('TFT8_Riven', 423), ('TFT8_Syndra', 437), ('TFT8_Zoe', 444), ('TFT8_Galio', 452), ('TFT8_Urgot', 496), ('TFT8_Fiddl

#### TỈ LỆ CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA MÓN ĐỒ TRONG TỪNG TOP CỦA TRẬN ĐẤU

In [102]:
all_item_df = pd.read_csv("process_data_champion_tft.csv", usecols=['Champion'])

In [103]:
all_items = all_item_df['Champion'].to_list()


In [104]:
sum = 0
temp = 0
item_avg_top = []
item_avg_on_match = []
match_each_top = []
for item in all_items:
    temp1 = []
    temp2 = []
    for i in range(1, 9, 1):
        cc = dict(item_on_top['top_' + str(i)][0])

        if(item in cc):
            temp = temp+dict(item_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(item_on_top['top_' + str(i)][0])[item])
            
        temp2.append(sum/32000)
        item_avg_on_match.append(temp2)
    temp1.append(temp/sum)
    item_avg_top.append(temp1)
    temp = 0
    sum = 0


TFT8_Alistar
TFT8_Alistar
TFT8_Alistar
TFT8_Alistar
TFT8_Alistar
TFT8_Alistar
TFT8_Alistar
TFT8_Alistar
TFT8_Annie
TFT8_Annie
TFT8_Annie
TFT8_Annie
TFT8_Annie
TFT8_Annie
TFT8_Annie
TFT8_Annie
TFT8_Aphelios
TFT8_Aphelios
TFT8_Aphelios
TFT8_Aphelios
TFT8_Aphelios
TFT8_Aphelios
TFT8_Aphelios
TFT8_Aphelios
TFT8_Ashe
TFT8_Ashe
TFT8_Ashe
TFT8_Ashe
TFT8_Ashe
TFT8_Ashe
TFT8_Ashe
TFT8_Ashe
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_AurelionSol
TFT8_BelVeth
TFT8_BelVeth
TFT8_BelVeth
TFT8_BelVeth
TFT8_BelVeth
TFT8_BelVeth
TFT8_BelVeth
TFT8_BelVeth
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Blitzcrank
TFT8_Camille
TFT8_Camille
TFT8_Camille
TFT8_Camille
TFT8_Camille
TFT8_Camille
TFT8_Camille
TFT8_Camille
TFT8_Chogath
TFT8_Chogath
TFT8_Chogath
TFT8_Chogath
TFT8_Chogath
TFT8_Chogath
TFT8_Chogath
TFT8_Chogath
TFT8_Draven
TFT8_Draven
TFT8_Draven
TFT8

In [105]:
print(item_avg_top[0], item_avg_on_match[0])


[4.26350632462466] [0.0388125, 0.07509375, 0.1094375, 0.14328125, 0.17565625, 0.2071875, 0.23825, 0.26434375]
